# Inferentia2 (inf2.48xlarge)에서 yanolja/KoSOLAR-10.7B-v0.2 배치 추론 

---




# 1. 사전 필수 단계
- 아래를 클릭하셔서 사전 단계를 수행 하세요.
    - [AWS Inferentia2 설치 및 실행 가이드](Readme.md)

# 2. 배치 추론 실행

In [2]:
from vllm import LLM , SamplingParams

# 3. yanolja/KoSOLAR-10.7B-v0.2 모델 컴파일 후에 로딩
- 아래의 파라미터에 대해서 기존에는 128 이었으나, 1024 로 변경하여 진행 함.
    - max_model_len=1024,
    - block_size=1024,

아래는 약 70 분이 소요 되었습니다.

In [3]:
llm = LLM(
    # model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model="yanolja/KoSOLAR-10.7B-v0.2",
    max_num_seqs=8,
    # The max_model_len and block_size arguments are required to be same as
    # max sequence length when targeting neuron device.
    # Currently, this is a known limitation in continuous batching support
    # in transformers-neuronx.
    # TODO(liangfu): Support paged-attention in transformers-neuronx.
    max_model_len=1024,
    block_size=1024,
    # The device can be automatically detected when AWS Neuron SDK is installed.
    # The device argument can be either unspecified for automated detection,
    # or explicitly assigned.
    device="neuron",
    tensor_parallel_size=2)

INFO 03-31 12:39:04 llm_engine.py:87] Initializing an LLM engine with config: model='yanolja/KoSOLAR-10.7B-v0.2', tokenizer='yanolja/KoSOLAR-10.7B-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cpu, seed=0)


2024-03-31 12:42:28.000186:  70201  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-31 12:42:28.000196:  70201  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/084afca8-7030-4d30-bde7-7ada13ca1f93/model.MODULE_9f050279deca50a4ec46+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/084afca8-7030-4d30-bde7-7ada13ca1f93/model.MODULE_9f050279deca50a4ec46+2c2d707e.neff --model-type=transformer --auto-cast=none --verbose=35
2024-03-31 12:42:28.000407:  70202  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-03-31 12:42:28.000470:  70202  INFO ||NEURON_CC_WRAPPER||: Call compiler with cmd: neuronx-cc compile --target=trn1 --framework=XLA /tmp/ubuntu/neuroncc_compile_workdir/06656340-372a-4b43-8ad6-1c5ebf6a6dbd/model.MODULE_d80e3715f30f9a482a22+2c2d707e.hlo_module.pb --output /tmp/ubuntu/neuroncc_compile_workdir/06656340-372a-

# 4. 모델 배치 추론

In [4]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [5]:
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s]

2024-Mar-31 13:47:31.0203 69535:75779 [1] nccl_net_ofi_init:1415 CCOM WARN NET/OFI aws-ofi-nccl initialization failed
2024-Mar-31 13:47:31.0203 69535:75779 [1] init.cc:137 CCOM WARN OFI plugin initNet() failed is EFA enabled?


Processed prompts: 100%|██████████| 4/4 [00:02<00:00,  1.48it/s]

Prompt: 'Hello, my name is', Generated text: ' Jaime and I am a Writer, Editor, and Social Media Manager.\n'
Prompt: 'The president of the United States is', Generated text: ' now speaking. He\'s delivering the final speech of his administration.\n"'
Prompt: 'The capital of France is', Generated text: ' home to about 2.2 million people.\nThe population is growing rapidly'
Prompt: 'The future of AI is', Generated text: ' in our hands: KLF19 Day 1 | GoKh'


In [6]:
prompts = [
    "대한민국의 수도는 어디야?",
    "사과의 건강 효능에 대해서 알려줘",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


In [7]:
outputs = llm.generate(prompts, sampling_params)
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 2/2 [00:01<00:00,  1.39it/s]

Prompt: '대한민국의 수도는 어디야?', Generated text: ' 그 질문에 대한 답이 서울이라면 이미 당신은 한국 역사의 뿌리'
Prompt: '사과의 건강 효능에 대해서 알려줘', Generated text: ' 줘\n사과의 건강 효능에 대해서 알려줘.\n중학교'
